<a href="https://colab.research.google.com/github/Yan-Weijia/CompLing/blob/master/HW03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Задание 1

In [0]:
import os, re # 文件路径库；字符串匹配库
from string import punctuation
import numpy as np # 维度数组和矩阵，针对数组运算的函数
import json # 数据交换格式 "key":value {}object无序 []array有序 value可以是""括起来的string, number, true, false, null, object, array
from collections import Counter
from pprint import pprint # 格式化输出
from nltk import sent_tokenize
punctuation += "«»—…“”"
punct = set(punctuation)

from sklearn.metrics import classification_report, accuracy_score # 机器学习算法库.矩阵运算；分类，精确度

In [2]:
import nltk
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
!wget https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/master/data/sents_with_mistakes.txt
!wget https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/master/data/correct_sents.txt

--2019-11-22 20:59:23--  https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/master/data/sents_with_mistakes.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 123167 (120K) [text/plain]
Saving to: ‘sents_with_mistakes.txt’

sents_with_mistakes 100%[===================>] 120.28K  --.-KB/s    in 0.04s   

2019-11-22 20:59:23 (3.07 MB/s) - ‘sents_with_mistakes.txt’ saved [123167/123167]

--2019-11-22 20:59:25--  https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/master/data/correct_sents.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awa

In [0]:
bad = open('sents_with_mistakes.txt', encoding='utf8').read().splitlines()
true = open('correct_sents.txt', encoding='utf8').read().splitlines()

In [5]:
!wget https://github.com/mannefedov/compling_nlp_hse_course/raw/master/data/corpus_500.txt

--2019-11-22 20:59:35--  https://github.com/mannefedov/compling_nlp_hse_course/raw/master/data/corpus_500.txt
Resolving github.com (github.com)... 192.30.253.113
Connecting to github.com (github.com)|192.30.253.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/master/data/corpus_500.txt [following]
--2019-11-22 20:59:36--  https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/master/data/corpus_500.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1181403 (1.1M) [text/plain]
Saving to: ‘corpus_500.txt’

corpus_500.txt      100%[===================>]   1.13M  --.-KB/s    in 0.08s   

2019-11-22 20:59:37 (14.5 MB/s) - ‘corpus_500.txt’ saved [118140

In [0]:
# txt里列表里每一句拆分成词 [[词，词，词，词]，[词，词，词]]
def norm_txt(text):
  new_txt = []
  for sent in text:
    tokens = sent.lower().split()
    tokens = [re.sub('(^\W+|\W+$)', '', tkn) for tkn in tokens if (set(tkn)-punct)]
    new_txt.append(tokens)
  return new_txt

In [0]:
badlist = norm_txt(bad)
truelist = norm_txt(true)

In [0]:
corpus = [] # [["",""],["",""],["",""]] 把词按句子分组
for text in open('corpus_500.txt').read().splitlines():
    sents = sent_tokenize(text)
    norm_sents = [normalize(sent) for sent in sents]
    corpus += norm_sents

In [0]:
WORDS = Counter() # corpus词出现的频次
for sent in corpus:
    WORDS.update(sent)
dict_corp = dict(WORDS) # 库词：频次

In [9]:
WORDS.most_common(10)

[('в', 4263),
 ('и', 1999),
 ('на', 1757),
 ('что', 1236),
 ('с', 990),
 ('по', 850),
 ('не', 739),
 ('он', 425),
 ('из', 406),
 ('этом', 405)]

In [0]:
# set去重corpus词
vocab = set() # set会去重，列表update到set：列表中的项添加到set

for sent in corpus:
    vocab.update(sent)


单个库词去掉n个字母的各种形式list

In [0]:
def edits(word):
    "Создаем кандидатов, которые отличаются на одну букву \
    找出差一个字母的选项"
    letters    = 'йцукенгшщзхъфывапролджэячсмитьбюё'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R] # 漏打一个字母
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1 and R[0] != R[1]] # 颠倒两字母顺序
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters] # 错打成别的字母
    inserts    = [L + c + R               for L, R in splits for c in letters] # 多打一个字母
    return list(set(deletes + transposes + replaces + inserts))

In [0]:
# 建dict_corp2 {错形式：[正确词], 错形式：[正确词, 正确词]}
dict_corp2 = {}
for word in dict_corp:
  for edword in edits(word):
    if edword not in dict_corp2:
      dict_corp2[edword] = [word]
    else:
      dict_corp2[edword].append(word)

In [30]:
st = set()
st.update([1,2])
st.update([1,2,3,4])
print(st)

{1, 2, 3, 4}


In [0]:
def edits1(word):
    "Создаем кандидатов, которые отличаются на одну букву \
    找出差一个字母的选项"
    letters    = 'йцукенгшщзхъфывапролджэячсмитьбюё'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R] # 漏打一个字母的错误选项
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1] # 颠倒两字母顺序的错误选项
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters] # 改一个字母的错误选项
    inserts    = [L + c + R               for L, R in splits for c in letters] # 加一个字母的错误选项
    return set(deletes + transposes + replaces + inserts + word.split())

In [0]:
def correction(badword):
  if badword in dict_corp:
    corrected = badword # 没错不用改
  else:
    bad_forms = edits1(badword) # {bad1, bad2, ...}
    variants = set()
    for form in bad_forms:
      if form not in dict_corp2:
        corrected = form # 此错库中无，或正好改对了
      else:
        variants.update(dict_corp2[form])
    dict_vars = {} # {候选：频次}
    for var in list(variants):
      dict_vars[var] = dict_corp[var]
    for var in dict_vars.keys():
      if dict_vars[var] == max(freq for freq in dict_vars.values()):
        corrected = var
  return corrected

In [64]:
%%time
print(correction('сонце'))

конце
CPU times: user 1.11 ms, sys: 1.02 ms, total: 2.13 ms
Wall time: 1.69 ms


In [65]:
%%time
correction('солнце')

CPU times: user 5 µs, sys: 1 µs, total: 6 µs
Wall time: 7.87 µs


'солнце'

In [66]:
%%time
correction('насмехатьсяаававттававаываываы')

CPU times: user 4.62 ms, sys: 0 ns, total: 4.62 ms
Wall time: 4.37 ms


'насмехатьсяаававттававадываываы'

In [0]:
# напишем функцию, которая будет сопоставлять слова в правильном и ошибочном варианте
# разобьем предложение по пробелам и удалим пунктуация на границах слов
# 匹配正确和错误的对(alignment)；按空格把句子拆分成词，去除标点
def align_words(sent_1, sent_2):
    tokens_1 = sent_1.lower().split()
    tokens_2 = sent_2.lower().split()
    
    tokens_1 = [re.sub('(^\W+|\W+$)', '', token) for token in tokens_1 if (set(token)-punct)]
    tokens_2 = [re.sub('(^\W+|\W+$)', '', token) for token in tokens_2 if (set(token)-punct)]
    
    return list(zip(tokens_1, tokens_2))

In [63]:
correct = 0 # 订正后对的总词数（本来对没动的 + 本来错改对的）
total = 0 # 总词数

total_mistaken = 0 # 总错词数
mistaken_fixed = 0 # 改对了数

total_correct = 0 # 本来是对的数
correct_broken = 0 # 改错了数

cashed = {}
for i in range(len(true)): # 正确词的数量
    word_pairs = align_words(true[i], bad[i]) # 对照配对
    for pair in word_pairs:
        total += 1 # 总词数
        predicted = cashed.get(pair[1], correction(pair[1])) # 错误词、词库修正后词配对字典
        cashed[pair[0]] = predicted # 正确词、词库修正后词
        if predicted == pair[0]: # 修正词 == 对词
            correct += 1 # 订正后对的总词数
        
        if pair[0] == pair[1]: # 对词 == 对错
            total_correct += 1 # 错词是对的数
            if pair[0] !=  predicted: # 对词 != 修正词
                correct_broken += 1 # 改错了数
        else: # 错词 != 对词
            total_mistaken += 1 # 总错词数
            if pair[0] == predicted: # 对词 == 修正词
                mistaken_fixed += 1 # 改对了数
        
    if not i % 100:
        print(i)
        

0
100
200
300
400
500
600
700
800
900


In [67]:
print(correct/total) # 订正后总对的数/总词数
print(mistaken_fixed/total_mistaken) # 改对了数/总错词数
print(correct_broken/total_correct) # 改错了数/本来是对的数

0.4944055944055944
0.1987720644666155
0.4613529344205811
